In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Tratamento dos Dados

In [ ]:
#lê-se os dados do arquivo csv
teste = pd.read_csv('/kaggle/input/airline-passenger-satisfaction/test.csv')
treino = pd.read_csv('/kaggle/input/airline-passenger-satisfaction/train.csv')

In [ ]:
teste.info()

In [ ]:
treino.info()

As duas bases de dados apresentam dados de uma pesquisa de opinião com usuários de uma empresa aérea. Grande parte das variáveis presentes dizem respeito a nota que os passageiros dão a determinados quesitos da experiência oferecida pela companhia, como é o caso das variáveis 'Gate Location' e 'Inflight wifi service', o que caracteriza esses dados como categóricos ordinais. Alguns outros dados são variáveis numéricas continuas, como é o caso de 'Arrival Delay in Minutes', 'Departure Delay in Minutes', 'Flight Distance'e 'Age'. A variável gênero é uma variável categorica nominal.

## Valores NAs

In [ ]:
# Observa-se o número de valores NAs presentes nas tabelas de dados.
treino.isnull().sum(), teste.isnull().sum()

Observando os valores NAs presentes nos dataframes providos, vê-se que existem um número reduzido deles, algo em torno de 0,3% do total de linhas, o que é um número bem reduzido. A princípio esses dados faltantes não são fator de preocupação, porém, quando analisar-se a multicolinearidade dos dados do dataframe, os NAs também serão tratados.

A seguir, as duas tabelas, de treino e de teste, são jutnadas em apenas um DF, o que irá facilitar o tratamento inicial dos dados. Logo em seguida, os dois DFs serão separados novamente.

In [ ]:
df = treino.append(teste)

df.info()

## Tratamento do nome das colunas

In [ ]:
#substitui-se os espaços presentes no título das colunas por 'underscore', o que facilita o lidar com as colunas mais a frente.
df.columns = [nome.replace(' ', '_') for nome in teste.columns]

df.head().T

Como as variáveis 'Unnamed:_0' e 'id' não serão utilizadas no modelo, pois uma representa um índice antigo do DF e a outra a identidade dos usuários que proveram as informações, elas serão retiradas do DF.

In [ ]:
df = df.drop(['Unnamed:_0', 'id'], axis = 1)

df.head().T

## Multicolinearidade

Abaixo é criada uma matriz de correlação entre as variáveis numéricas restantes no DF, para saber se existem variáveis com problemas de multicolinearidade e ver quais mais se correlacionam entre si.

In [ ]:
fig = plt.figure(figsize = (12,10))
sns.heatmap(df.corr(), vmin = -1, vmax = 1, annot = True, fmt = '.2f')
plt.show()

Da matriz de correlação acima, percebe-se que as colunas 'Departure_Delay_in_Minutes' e 'Arrival_Delay_in_Minutes' estão com uma alta correlação entre si (97%). Sabe-se que um avião que parte atrasado, tende na maioria das vezes a também chegar atrasado. 
Além de ser afetado por multicolinearidade, a coluna 'Arrival_Delay_in_Minutes' apresenta 393 valores NaN, menos de 0,1% do total.

Dessa forma, opta-se por retirar a coluna 'Arrival_Delay_in_Minutes' para evitar-se problemas de multicolinearidade entre as colunas mencionadas anteriormente e livrar-se o dataframe de valores NaN.

In [ ]:
df = df.drop('Arrival_Delay_in_Minutes', axis = 1)

df.head().T

In [ ]:
df.isnull().sum()

Após tratar o nome das colunas, valores NAs, multicolinearidade, o DF, criado juntando os dados de treino e teste, é dividido novamente em tabelas de treino e teste com as mesmas dimensoes iniciais.

In [ ]:
treino = df.iloc[:103904,:]
teste = df.iloc[103904:,:]

In [ ]:
treino.info(), teste.info()

Abaixo, confirma-se que os valores NAs foram retirados das bases de dados.

In [ ]:
teste.isnull().sum(), treino.isnull().sum()

## Outliers

Deve-se procurar, identificar e tratar os valores outliers presentes nas bases, uma vez que esses valores podem induzir os modelos de ML ao erro, já que são dados aberrantes. 

Para isso, foi criada uma função capaz de plotar o Boxplot de todas as variáveis dos DFs. Esses Boxplots permitem que os outliers sejam identificados visualmente para em seguida serem tratados.

In [ ]:
#Plota-se o BoxPlot de todas as colunas númericas, para observar-se a existência de outliers
def boxplot(df):
    
    fig = plt.figure(figsize=(15, 12))


    #loop para percorrer e plotar valores de todas as colunas do DF
    for col in range(1, df.shape[1]):
        if df.iloc[:, col].dtype == 'int64' or df.iloc[:, col].dtype == 'float64':
            plt.subplot(4,8, col)
            f = plt.gca()
            f.axes.get_yaxis().set_visible(False)

            plt.boxplot(df.iloc[:, col])
            plt.title(df.columns[col]) 

    plt.tight_layout()

In [ ]:
#Boxplot do DF 'teste'
boxplot(teste)

In [ ]:
#Boxplot do DF 'treino'
boxplot(treino)

Dos Boxplots, precebe-se que as bases de teste e de treino possuem outliers nas mesmas variáveis: 'Flight_Distance' e 'Departure_delay_in_minutes'.

A partir dessa identificação, torna-se possível o tratamento desses valores, que será feito pela retirada desses valores das bases obedecendo as distâncias interqualicas, que determinam o que é e o que não é outliers.

In [ ]:
#Cálculo do IQR para precipitação, que será usado para identicar e filtrar os outliers para o DF de Teste

iqr = teste.Flight_Distance.quantile(0.75) - teste.Flight_Distance.quantile(0.25)
iqr_Flight_Distance = teste.Flight_Distance.quantile(0.75) + (iqr * 1.5)

iqr = teste.Departure_Delay_in_Minutes.quantile(0.75) - teste.Departure_Delay_in_Minutes.quantile(0.25)
iqr_Departure_Delay_in_Minutes = teste.Departure_Delay_in_Minutes.quantile(0.75) + (iqr * 1.5)

iqr = teste.Checkin_service.quantile(0.75) - teste.Checkin_service.quantile(0.25)
iqr_Checkin_service = teste.Checkin_service.quantile(0.25) - (iqr * 1.5)

#Filtro de outliers
teste_sem_outliers = teste[teste['Flight_Distance']<iqr_Flight_Distance]
teste_sem_outliers = teste_sem_outliers[teste_sem_outliers['Departure_Delay_in_Minutes']<iqr_Departure_Delay_in_Minutes]
teste_sem_outliers = teste_sem_outliers[teste_sem_outliers['Checkin_service']>iqr_Checkin_service]

boxplot(teste_sem_outliers)

In [ ]:
#Cálculo do IQR para precipitação, que será usado para identicar e filtrar os outliers

iqr = treino.Flight_Distance.quantile(0.75) - treino.Flight_Distance.quantile(0.25)
iqr_Flight_Distance = treino.Flight_Distance.quantile(0.75) + (iqr * 1.5)

iqr = treino.Departure_Delay_in_Minutes.quantile(0.75) - treino.Departure_Delay_in_Minutes.quantile(0.25)
iqr_Departure_Delay_in_Minutes = treino.Departure_Delay_in_Minutes.quantile(0.75) + (iqr * 1.5)

iqr = treino.Checkin_service.quantile(0.75) - treino.Checkin_service.quantile(0.25)
iqr_Checkin_service = treino.Checkin_service.quantile(0.25) - (iqr * 1.5)

#Filtro de precipitação
treino_sem_outliers = treino[treino['Flight_Distance']<iqr_Flight_Distance]
treino_sem_outliers = treino_sem_outliers[treino_sem_outliers['Departure_Delay_in_Minutes']<iqr_Departure_Delay_in_Minutes]
treino_sem_outliers = treino_sem_outliers[treino_sem_outliers['Checkin_service']>iqr_Checkin_service]

boxplot(treino_sem_outliers)


Como a distância interquartílica é calculada baseada nos dados presentes nas bases, ainda é possível ver que alguns valores que a princiípio não eram outliers passaram a ser considerados outliers. Isso ocorre porque, com a retirada de alguns valores heterogêneos, os dados passam a ser mais homogêneos entre sim, diminui-se a distância interquartílica. Quando isso ocorre, valores que antes estavam dentro dessa distância, passam a ficar fora dela, afinal ela foi reduzida. 

Mesmo ainda presentes outliers nas bases, resolveu-se não fazer mais uma retirada de dados para que não houvesse uma perda de dados relevante mas bases. Entretanto, é possível perceber como o perfil dos Boxplots mudou, quando comparado com os mesmos com os valores outliers.

In [ ]:
treino.head().T

# Análise Explanatória

No próximo passo, são selecionadas as colunas com valores numéricos discretos para que sejam plotados em função da satisfação dos usuários. Logo após, as outras colunas, com valores numericos contínuos serão plotadas em forma de histograma. 

A variável 'satisfation' não foi plotada pois trata-se da variável target do problema.

In [ ]:
#Define-se as que não serão plotadas em gráficos de barras.
exc_cols_plot = ['Age', 'Flight_Distance', 'Departure_Delay_in_Minutes', 'satisfaction']

#Seleciona-se as variáveis que serão plotadas em gráficos de barras
cols_plot = [c for c in treino_sem_outliers.columns if c not in exc_cols_plot]

#Plota-se os gráficos:
fig, ax = plt.subplots(6, 3, figsize=(20, 30))
for variable, subplot in zip(cols_plot, ax.flatten()):
    sns.countplot(data = treino_sem_outliers,
                  x = variable,
                  hue=treino_sem_outliers['satisfaction'], 
                  ax=subplot).set_title(f'Distribuição da Satisfação por {variable}')

Dos gráficos plotados acima, chega-se as seguintes conclusões:
* Existem mais mulheres insatisfeitas que homens;
* Consumidores leais são maioria entre os satisfeitos;
* Passageitos de negócios são mais satisfeitos que os viajantes a lazer;
* É possível que haja uma relação entre consumidores de negócios e consumidores de leais;
* Viajantes das classes econômicas são mais insatisfeitos que os de classe Executiva;
* Passageiros que atribuem notas 4 e 5 aos serviços oferecidos pela companhia aérea tendem a ser mais satisfeitos;
* Alguns serviços ó tem clientes satisfeitos quando a nota é 5, que o caso de serviços durante o voo, serviço de checkin e localização do portão de embarque;
* A variável que trata da conveniência do horário de partida e chegada dos voos apresenta mais clientes insatisfeitos nas notas 4 e 5.


In [ ]:
continuous_cols_plot = ['Age', 'Flight_Distance', 'Departure_Delay_in_Minutes']

fig, ax = plt.subplots(1, 3, figsize=(20, 10))
for variable, subplot in zip(continuous_cols_plot, ax.flatten()):
    sns.histplot(data = treino_sem_outliers,
                 x = treino_sem_outliers[variable],
                 bins = 10,
                 hue=treino_sem_outliers['satisfaction'], 
                 ax=subplot).set_title(f'Histograma de {variable}')

Quando fala-se em variáveis numéricas contínuas, percebe-se os seguintes pontos:
* Pessoas mais entre 40 e 60 anos são mais satisfeitas;
* Pessoas até 40 anos são mais insatisfeitas;
* A partir de 60 anos a pessoas são mais insatisfeitas também;
* Quanto mais longo o voo mais as pessoas são satisfeitas;
* Quando o voo atrasa a maior parte das pessoas tendem a ficar insatisfeitas.

Diante desses dados, pode-se traçar algumas hipóteses que justificam esses achados:
* Pessoas com voos mais longos tendem a comprar passagens na primeira classe para que tenham mais conforto durante o longo voo. Como a primeira classe tende a oferecer melhores serviços, como assentos mais confortáveis, comida e bebida, mais espaço para as pernas, os passageiros dessa classe tendem a ser mais satisfeitos por conta da melhor qualidade dos serviços oferecidos.
* Pessoas viajando a trabalho, normalmente no auge de sua idade produtiva, entre 40 e 60 anos, acabam sendo menos sensíveis a variações de preço na hora de comprar suas passagens. Portanto, podem acabar decidindo comprar passagens de primeira classe para o seu deslocamento.

É salutar notar que um anota baixa em um determinado item avaliado, não quer dizer necessáriamente que o passageiro ficara insatisfeito com a experiência inteira que a companhia aérea ofereceu-lhe, mas é um bom indicativo do grau de satisfação do cliente.

# Modelagem dos Dados

Ao modelar os dados usando algoritmos de Machine Learning, os dados categóricos 'Gender','Customer_Type', 'Type_of_Travel' e 'Class' devem ser transformados em variáveis Dummies. A função 'pd.Dummies' faz essa transformação e atribui valores 0 e 1 para as novas colunas criadas com base nos valores presentes em cada coluna original categórica.

In [ ]:
treino_com_dummies = pd.get_dummies(treino_sem_outliers, columns=['Gender', 
                                                                  'Customer_Type', 
                                                                  'Type_of_Travel',
                                                                  'Class'])

teste_com_dummies = pd.get_dummies(treino_sem_outliers, columns=['Gender', 
                                                                  'Customer_Type', 
                                                                  'Type_of_Travel',
                                                                  'Class'])

## Balanceamento dos Dados

In [ ]:
treino_com_dummies['satisfaction'].value_counts(), teste_com_dummies['satisfaction'].value_counts()

Observando a quantidade de dados disponíveis da variável target, percebe-se que tanto os dados de treino como os de teste apresentam mais dados informando sobre qualificações como 'neutral or dissatisfied' (56%) do que qualificações 'satisfied'(43%). Esse desbalanceamento dos dados pode deixar os modelos de Machine Learning enviesados para aquelas qualificações. A explicação é que o modelo estará exposto a mais dados de usuários insatisfeito do que dados de usuários satisfeito, portanto o modelo terá mais informações para classificar usuários como insatisfeitos.

Para resolver esse problema será utilizado um método de 'UnderSampling' para que se obtenha DFs de mesmo tamanho para ser usado nos modelos de ML.

In [ ]:
exc_cols_dt = ['satisfaction']
cols_dt = [ c for c in treino_com_dummies.columns if c not in exc_cols_dt]

cols_dt

In [ ]:
treino_com_dummies[cols_dt].head()

In [ ]:
from imblearn.under_sampling import RandomUnderSampler
#Instanciando o objeto RandomUnderSampler()
rus = RandomUnderSampler(random_state = 0)

#Balanceando a amostra
x_treino, y_treino = rus.fit_resample(treino_com_dummies[cols_dt], treino_com_dummies['satisfaction'])

x_treino.shape, y_treino.shape

In [ ]:
y_treino.value_counts()

Após aplicação da função 'RandomUnderSampler', os dados ficaram balanceados com 35429 usuários satisfeitos e 35429 usuários insatisfeitos.

## Random Forest

In [ ]:
# Importa-se a função que divide os dados em dados de treino e dados de teste para serem usados no modelo de ML
from sklearn.model_selection import train_test_split

In [ ]:
# Dividindo o dataframe
# Por padrão a divisão é de 75% para o conjunto maior (treino) e 25% para o menor (validação)
x_treino, x_valid, y_treino, y_valid = train_test_split(x_treino, y_treino, random_state = 42)

In [ ]:
#Confere-se o tamanho dos DFs utilizados
x_treino.shape, x_valid.shape, y_treino.shape, y_valid.shape

In [ ]:
#importa o modelo Random Forest Classifier
from sklearn.ensemble import RandomForestClassifier

#Instancia-se o modelo c/ 200 árvores na floresta e com uma uqatidade mínima de 3 folhas por árvore.
rf = RandomForestClassifier(n_estimators = 200, random_state = 42, n_jobs = -1, min_samples_leaf=3)

In [ ]:
#Treina-se o modelo com os dados de treino
rf.fit(x_treino, y_treino)

In [ ]:
# Previsão dos dados de validação com o modelo criado anteriormente
preds_val_rf = rf.predict(x_valid)

preds_val_rf

### Avaliando o desempenho do modelo Random Forest


In [ ]:
# Importando a metrica
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
#Calcula-se a acuracia das previsões de validação
acc_val_rf = accuracy_score(y_valid, preds_val_rf)

acc_val_rf

In [ ]:
#Calcula-se a acuracia nos dados de teste
preds_test_rf = rf.predict(teste_com_dummies[cols_dt])

acc_test_rf = accuracy_score(teste_com_dummies['satisfaction'], preds_test_rf)
acc_test_rf

In [ ]:
# Avaliando a importancia de cada coluna (cada variável de entrada)
ft_import_rf = pd.Series(rf.feature_importances_, index=cols_dt).sort_values().plot.barh()

In [ ]:
# importando a bilbioteca para plotar o gráfico de Matriz de Confusão
import scikitplot as skplt

In [ ]:
# Matriz de Confusão - Dados de Teste
conf_rf = skplt.metrics.plot_confusion_matrix(teste_com_dummies['satisfaction'], preds_test_rf, normalize=True)

## Decision Tree


In [ ]:
# Importando modelo e instanciando a árvore de decisão
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state = 42)

In [ ]:
#Treina-se o modelo de Decision Tree
dt.fit(x_treino, y_treino)

In [ ]:
# Previsão dos dados de validação com o modelo criado anteriormente
preds_val_dt = dt.predict(x_valid)

preds_val_dt

## Metrica para avaliar o Modelo

In [ ]:
#Calcula-se a acuracia das previsões de validação
acc_val_dt = accuracy_score(y_valid, preds_val_dt)
acc_val_dt

In [ ]:
#Calcula-se a acuracia nos daods de teste
preds_test_dt = dt.predict(teste_com_dummies[cols_dt])

acc_test_dt = accuracy_score(teste_com_dummies['satisfaction'], preds_test_dt)
acc_test_dt

In [ ]:
# Avaliando a importancia de cada coluna (cada variável de entrada)
ft_import_dt = pd.Series(dt.feature_importances_, index=cols_dt).sort_values().plot.barh()

In [ ]:
# Matriz de Confusão - Dados de Validação
conf_dt = skplt.metrics.plot_confusion_matrix(teste_com_dummies['satisfaction'], preds_test_dt, normalize=True)

## Comparando os Modelos

Após rodar os modelos de Decision Tree e Random Forest com 200 árvores e um mínimo de 3 folhas, obteve-se as métricas para que os modelos fossem avaliados e verificar-se quais dos modelos permite uma melhor predição da satisfação dos usuários da companhia área.

In [ ]:
pd.Series(dt.feature_importances_, index=cols_dt).sort_values().plot.barh()

In [ ]:
pd.Series(rf.feature_importances_, index=cols_dt).sort_values().plot.barh()

Primeiramente, é interessante notar que ambos os modelos produzem suas predições baseados em pesos diferentes atribuidos a colunas diferentes. O modelo de Decision Tree atribuiu maiores pesos a 'Online_Boarding', 'Inflight_Wifi_Service, o mesmo que aconteceu com o modelo de Random Forest, porém com maior peso na última variável. A partir do terceiro maior peso atribuido pelo modelo, há diferenças entre os modelos. Enquanto o modelo de Decision Tree teve como terceiro maior peso 'Type of_travel_Business_travel', o modelo de Random Forest deu maior peso a variável 'Class_Business'. 

In [ ]:
acc_test_rf, acc_test_dt

Quando fala-se da acurácia dos dois modelos, percebe-se que o modelo de Decision Tree leva uma ligeira vantagem sobre o modelo de Random Forest quando aplicados sobre os dados de teste fornecidos. Ou seja, o primeiro modelo acertou mais previsoões que o segundo.

Quando utiliza-se uma matriz de confusão quer-se visualizar o desempenho dos augoritmos de classificação usados. Abaixo apresenta-se a matriz de confusão feita para cada um dos modelos, onde poderar-se ver os erros de previsão. Aqui faz-se necessário comentar que dados desbalanceados, como eram os usados nesse trabalho antes de serem balanceados acima, produz dados enganosos.

In [ ]:
skplt.metrics.plot_confusion_matrix(teste_com_dummies['satisfaction'], preds_test_dt, normalize=True), skplt.metrics.plot_confusion_matrix(teste_com_dummies['satisfaction'], preds_test_rf, normalize=True)

Os dados da matriz de previsão permitem perceber que os modelos produzem resultados muito similares, porém o modelo de Decision Tree leva ligeira vantagem quando fala-se na condição negativa prevista, pois ele apresenta maior acerto ao predizer clientes satisfeitos que realmente estão satisfeitos e menor erro ao prever clientes insatisfeitos quando eles estão satisfeitos, erro de falso negativo (Erro do Tipo II).

Essa vantagem é especialmente interessante para a companhia área pois permite que ela economize recursos, já que ela pode deixar de fazer campanhas para melhorar a satisfação do cliente para clientes que já estão satisfeitos. 

# Conclusão


Embora, o modelo de Decision Tree leve ligeira vantagem, os dois modelos são bastante equivalente, e essa pequena vantagem pode ser atribuida ao aleatório, a metodologia utilizada no tratamento dos dados e até mesmo aos dados selecionados para fazerem parte de cada set de dados para treino e teste.

Para concluir, a companhia pode utilizar qualquer um dos dois modelos para prever a satisfação dos clientes. 

A utilização de modelos de ML pode deixar a companhia mais eficiente em seus planos de marketing, permitindo alocar melhor os recursos e impactar os clientes que realmente devem ser impactados para trazerem melhores resultados financeiros para a companhia.

Com os modelos e resultados apresentados acima, pode-se deixar como sugestão, pede-se que a companhia aérea atente-se mais aos serviços de  'Online_Boarding' e 'Inflight_Wifi_Service e também tente melhorar a experiência dos usuários de 'Type of_travel_Business_travel' e 'Class_Business', já que essas quatro variáveis são as que mais impactam a satisfação dos usuários independente do modelo de predição utilizado.